# Part 2: Data Munging

In [49]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

## Open Webscraped Data

In [50]:
with open("countries_data_scraped.pkl", 'rb') as picklefile: 
    countries_data = pickle.load(picklefile)

In [51]:
countries_data.head()

,Country,url,Change in rank,Score,Change in score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,...,Life_Expectancy,Fertility_Rate,Hospital_Bed_Density,Access_To_Clean_Water,Obesity_Rate,School_Life_Expectancy,GDP,GDP_per_capita,Unemployment_Rate,Electrification_Rate
0,Afghanistan,geos/af.html,13,3.794,0.434,0.401,0.582,0.181,0.106,0.312,...,51.7,5.12,0.5,0.553,0.055,11.0,6.951000e+10,1900.0,0.350,0.430
2,Albania,geos/al.html,0,4.644,-0.011,0.996,0.804,0.731,0.381,0.201,...,78.5,1.51,2.6,0.836,0.217,16.0,3.587000e+10,12500.0,0.140,1.000
3,Algeria,geos/ag.html,-15,5.872,-0.483,1.092,1.146,0.618,0.233,0.069,...,77.0,2.70,NaN,0.836,0.274,14.0,6.293000e+11,15100.0,0.117,0.990
6,Angola,geos/ao.html,1,3.795,-0.071,0.858,1.104,0.050,0.000,0.098,...,60.2,6.16,NaN,0.490,0.082,10.0,1.920000e+11,6800.0,NaN,0.300
11,Argentina,geos/ar.html,2,6.599,-0.051,1.185,1.440,0.695,0.495,0.109,...,77.3,2.26,4.7,0.991,0.283,17.0,9.115000e+11,20700.0,0.081,0.964


## Data Munging

In [52]:
countries_data = countries_data.reset_index(drop=True)

In [53]:
countries_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 28 columns):
Country                         150 non-null object
url                             150 non-null object
Change in rank                  150 non-null object
Score                           150 non-null object
Change in score                 150 non-null float64
GDP per capita                  150 non-null float64
Social support                  150 non-null float64
Healthy life expectancy         150 non-null float64
Freedom to make life choices    150 non-null float64
Generosity                      150 non-null float64
Trust                           150 non-null float64
Residual                        150 non-null float64
Population                      149 non-null float64
Land_Area                       148 non-null float64
Support_Ratio                   149 non-null float64
Urbanization                    148 non-null float64
Median_Mothers_Age              114 non-null float6

Drop a few of the columns that we know we aren't interested in.  

We'll use data from the World Fact Book in place of the World Happiness Report's `GDP per capita` and `Healthy life expectancy`.

`Change in score`, `Change in rank` and `Residual` were all calculated in the World Happiness Report and have weird relationships with `Score`.




In [54]:
countries = countries_data.drop(["Change in score",'Change in rank','url', 'GDP per capita','Residual', 'Healthy life expectancy'], axis = 1)

Make sure all the entries in `Score` are numeric.

In [55]:
countries['Score'] = countries['Score'].apply(pd.to_numeric)

Create `Population Density` from `Population` and `Land_Area`.

In [56]:
countries['Population Density'] = countries['Population']/countries['Land_Area']

Look at the percentage of NaNs per column.

In [57]:
countries.isnull().sum()/150

Country                         0.000000
Score                           0.000000
Social support                  0.000000
Freedom to make life choices    0.000000
Generosity                      0.000000
Trust                           0.000000
Population                      0.006667
Land_Area                       0.013333
Support_Ratio                   0.006667
Urbanization                    0.013333
Median_Mothers_Age              0.240000
Get_Median_Age                  0.000000
Life_Expectancy                 0.020000
Fertility_Rate                  0.013333
Hospital_Bed_Density            0.086667
Access_To_Clean_Water           0.020000
Obesity_Rate                    0.020000
School_Life_Expectancy          0.126667
GDP                             0.000000
GDP_per_capita                  0.006667
Unemployment_Rate               0.046667
Electrification_Rate            0.006667
Population Density              0.020000
dtype: float64

Drop `Median_Mothers_Age` because 24% of the values are NaNs.

In [58]:
countries = countries.drop(['Median_Mothers_Age'],axis = 1)

Fill `NaN`s with the mean of that column.

In [59]:
countries = countries.fillna(countries.mean())

In [60]:
with open('countries_data_munged.pkl', 'wb') as picklefile: # wb: write, binary
    pickle.dump(countries, picklefile) #dump data into pickle file

In [61]:
countries.head()

,Country,Score,Social support,Freedom to make life choices,Generosity,Trust,Population,Land_Area,Support_Ratio,Urbanization,...,Fertility_Rate,Hospital_Bed_Density,Access_To_Clean_Water,Obesity_Rate,School_Life_Expectancy,GDP,GDP_per_capita,Unemployment_Rate,Electrification_Rate,Population Density
0,Afghanistan,3.794,0.582,0.106,0.312,0.061,34124811.0,652230.0,21.2,0.276,...,5.12,0.500000,0.553,0.055,11.0,6.951000e+10,1900.0,0.350000,0.430,52.320211
1,Albania,4.644,0.804,0.381,0.201,0.040,3047987.0,27398.0,5.5,0.593,...,1.51,2.600000,0.836,0.217,16.0,3.587000e+10,12500.0,0.140000,1.000,111.248522
2,Algeria,5.872,1.146,0.233,0.069,0.146,40969443.0,2381741.0,11.2,0.719,...,2.70,3.089781,0.836,0.274,14.0,6.293000e+11,15100.0,0.117000,0.990,17.201469
3,Angola,3.795,1.104,0.000,0.098,0.070,29310273.0,1246700.0,21.9,0.456,...,6.16,3.089781,0.490,0.082,10.0,1.920000e+11,6800.0,0.112098,0.300,23.510286
4,Argentina,6.599,1.440,0.495,0.109,0.060,44293293.0,2736690.0,5.8,0.920,...,2.26,4.700000,0.991,0.283,17.0,9.115000e+11,20700.0,0.081000,0.964,16.184987


In [64]:
countries.to_csv('countries.csv', encoding='utf-8', index=False)